In [1]:
# ! pip install OpenNMT-py

In [2]:
# ! pip install rouge

In [ ]:
#!pip install --upgrade https://storage.googleapis.com/jax-releases/cuda102/jaxlib-0.1.44-cp37-none-linux_x86_64.whl

In [3]:
# !pip install --upgrade jax

In [ ]:
#! pip install trax

In [1]:
import onmt
import pyonmttok
import torch
import json
import re
import os
import numpy as np
from tqdm import tqdm
import rouge
import gin
import trax
import jax

### 0. Загрузка

In [0]:
#! wget https://github.com/RossiyaSegodnya/ria_news_dataset/raw/master/ria.json.gz

In [5]:
#! gunzip ria.json.gz

In [0]:
#TAG_RE = re.compile(r'<[^>]+>')

In [2]:
def remove_tags(text):
    clean = re.compile('<.*?>')
    droped_html_tags = re.sub(clean, ' ', text)
    return re.sub(r'\n|\r', ' ', droped_html_tags)

In [3]:
test_val_indexes = np.random.choice(1003869, 40000, replace=False) # 40000 индексов для теста и валидации

with open('ria.json') as f:
    for i, line in tqdm(enumerate(f)):
        l = json.loads(line)
        text = remove_tags(l['text'])
        title = remove_tags(l['title'])
        if i in test_val_indexes:
            with open('texts_val.txt', 'a') as text_file:
                text_file.write(text)
                text_file.write('\n')
            with open('labels_val.txt', 'a') as label_file:
                label_file.write(title)
                label_file.write('\n')
        else:
            with open('texts.txt', 'a') as text_file:
                text_file.write(text)
                text_file.write('\n')
            with open('labels.txt', 'a') as label_file:
                label_file.write(title)
                label_file.write('\n')

1003869it [02:55, 5733.59it/s]


In [4]:
with open("texts_val.txt") as f:
    lines = f.readlines()
    with open("val_texts.txt", "w") as f1:
        f1.writelines(lines[:20000])
    with open("test_texts.txt", "w") as f2:
        f2.writelines(lines[20000:])

In [5]:
with open("labels_val.txt") as f:
    lines = f.readlines()
    with open("val_labels.txt", "w") as f1:
        f1.writelines(lines[:20000])
    with open("test_labels.txt", "w") as f2:
        f2.writelines(lines[20000:])

### 1. BPE

In [4]:
# ~10 минут
tokenizer = pyonmttok.Tokenizer("conservative", joiner_annotate=True)
learner = pyonmttok.BPELearner(tokenizer=tokenizer, symbols=50000)
learner.ingest_file("texts.txt")
BPE = learner.learn("model-50k_with_joiner", verbose=True)

In [8]:
os.mkdir('lstm_v2')

In [5]:
# ~ 20 минут
BPE.tokenize_file(input_path='texts.txt', output_path='lstm_v2/texts_tokenized_train')
BPE.tokenize_file(input_path='labels.txt', output_path='lstm_v2/labels_tokenized_train')

In [6]:
BPE.tokenize_file(input_path='val_texts.txt', output_path='lstm_v2/texts_tokenized_val')
BPE.tokenize_file(input_path='val_labels.txt', output_path='lstm_v2/labels_tokenized_val')
BPE.tokenize_file(input_path='test_texts.txt', output_path='lstm_v2/texts_tokenized_test')
BPE.tokenize_file(input_path='test_labels.txt', output_path='lstm_v2/labels_tokenized_test')

In [59]:
os.remove('texts.txt')
os.remove('labels.txt')
os.remove('val_texts.txt')
os.remove('val_labels.txt')
os.remove('test_texts.txt')
os.remove('test_labels.txt')
os.remove('texts_val.txt')
os.remove('labels_val.txt')
os.remove('lstm_v2/texts_tokenized_train')
os.remove('lstm_v2/labels_tokenized_train')
os.remove('lstm_v2/texts_tokenized_val')
os.remove('lstm_v2/labels_tokenized_val')
os.remove('lstm_v2/texts_tokenized_test')
os.remove('lstm_v2/labels_tokenized_test')

#### очистка от пустых строк после BPE

In [54]:
with open('lstm_v2/labels_tokenized_test') as f:
    lines = f.readlines()

In [50]:
nulls = []
for i, line in enumerate(lines):
    if line == '\n':
        nulls.append(i)

In [55]:
len(lines)

20000

In [57]:
for i, line in tqdm(enumerate(lines)):
    if i not in nulls:
        with open('lstm_v2/clear_labels_tokenized_test', 'a') as f:
            f.write(line)

20000it [00:00, 61913.21it/s]


### 2. Препроцессинг

In [58]:
#os.mkdir('lstm_v2/preprocessed_data')

In [61]:
os.listdir('lstm_v2')

['clear_texts_tokenized_val',
 'clear_labels_tokenized_train',
 'clear_labels_tokenized_val',
 'clear_texts_tokenized_test',
 'clear_texts_tokenized_train',
 'preprocessed_data',
 'clear_labels_tokenized_test']

In [ ]:
! onmt_preprocess -train_src lstm_v2/clear_texts_tokenized_train \
                  -train_tgt lstm_v2/clear_labels_tokenized_train \
                  -valid_src lstm_v2/clear_texts_tokenized_val \
                  -valid_tgt lstm_v2/clear_labels_tokenized_val \
                  -src_seq_length 3000 \
                  -tgt_seq_length 1000 \
                  -filter_valid \
                  -lower \
                  -shard_size 50000 \
                  -num_threads 6 \
                  -save_data lstm_v2/preprocessed_data/preprocessed_data

### 3. lstm

In [ ]:
!onmt_train -data lstm_v2/preprocessed_data/preprocessed_data  \
            -save_model lstm_v2/lstm_model_v2 \
            -train_steps 60000 \
            -valid_steps 10000 \
            -optim adam \
            -learning_rate 0.001 \
            -report_every 2000 \
            -gpu_ranks 0 \
            -tensorboard

In [64]:
! onmt_translate -gpu 0 \
               -batch_size 8 \
               -beam_size 10 \
               -model lstm_v2/lstm_model_v2_step_55000.pt \
               -src lstm_v2/clear_texts_tokenized_test \
               -output lstm_v2/pred_test.txt \
               -report_time \
               -min_length 1 \
               -stepwise_penalty \
               -coverage_penalty summary \
               -beta 5 \
               -length_penalty wu \
               -alpha 0.9 \
               -block_ngram_repeat 3

[2020-01-31 14:55:57,690 INFO] Translating shard 0.
PRED AVG SCORE: -0.2222, PRED PPL: 1.2488
Total translation time (s): 381.589313
Average translation time (s): 0.038159
Tokens per second: 262.900445
[2020-01-31 15:02:20,377 INFO] Translating shard 1.
PRED AVG SCORE: -0.2205, PRED PPL: 1.2467
Total translation time (s): 379.849602
Average translation time (s): 0.037985
Tokens per second: 267.390039


In [66]:
BPE.detokenize_file(input_path='lstm_v2/pred_test.txt', output_path='lstm_v2/pred_test_detokenized.txt')

In [68]:
BPE.detokenize_file(input_path='lstm_v2/clear_labels_tokenized_test', output_path='lstm_v2/true_test_detokenized.txt')

#### Очистка от пустых настоящих заголовков

In [101]:
with open('lstm_v2/pred_test_detokenized.txt') as f1:
    predictions = f1.readlines()
with open('lstm_v2/true_test_detokenized.txt') as f2:
    true = f2.readlines()

In [102]:
for i, element in enumerate(true):
    if len(element) < 5:
        print(i, element)

2612 ...



In [104]:
for i, element in enumerate(true):
    if i != 2612:        
        with open('lstm_v2/true.txt', 'a') as f1:
            f1.write(element)

In [105]:
for i, element in enumerate(predictions):
    if i != 2612:        
        with open('lstm_v2/predictions.txt', 'a') as f2:
            f2.write(element)

#### Метрики

In [106]:
from rouge import FilesRouge
files_rouge_detoc = FilesRouge('lstm_v2/predictions.txt', 'lstm_v2/true.txt')
scores_detoc = files_rouge_detoc.get_scores(avg=True)
scores_detoc

{'rouge-1': {'f': 0.40704861192204495,
  'p': 0.4254433360751212,
  'r': 0.39838380926281153},
 'rouge-2': {'f': 0.22824682595138046,
  'p': 0.23940092178879546,
  'r': 0.22377283102694753},
 'rouge-l': {'f': 0.3785419590169131,
  'p': 0.40359357600470086,
  'r': 0.3778888807812343}}

#### Сравнение предсказанных заголовков и настоящих

In [107]:
true[:30]

['греция попросила помощи ес в борьбе с лесными пожарами\n',
 'российские инспекторы совершат наблюдательные полеты над сша и канадой\n',
 'глушаков выйдет в стартовом составе сборной рф на матч против греции\n',
 'чуров ради ордена александра невского отказался от другой награды\n',
 'митрохин снова стал председателем "яблока"\n',
 'второй за день взрыв прогремел в багдаде, более 10 человек погибли\n',
 'заработал взломанный накануне сайт поддержки егэ в петербурге\n',
 'лавров обсудил с аннаном перспективы проведения конференции по сирии\n',
 'беспорядки вспыхнули в колонии на ставрополье\n',
 'власти испании позитивно восприняли доклад мвф\n',
 'биография игоря зубова\n',
 'замглавы мвд алешин снят с должности\n',
 '"новая голландия" 16 июня вновь открывается для посетителей\n',
 'выборы губернатора в новгородской области обойдутся в 59 млн руб\n',
 'дело заведено на возможного виновника дтп, где погиб полицейский\n',
 'константин зырянов пропустил тренировку сборной россии из-за пр

In [108]:
predictions[:30]

['греция попросила ес помочь в борьбе с лесными пожарами\n',
 'российские военные инспекторы совершат наблюдательные полеты над сша\n',
 'футболист глушаков выйдет в стартовом составе сборной россии по футболу\n',
 'чуров попросил наградить александра невского орденом александра невского\n',
 '"яблоко" переизбрал митрохина на пост председателя партии\n',
 'число погибших при взрыве в багдаде возросло до 11 человек\n',
 'сайт поддержки егэ, взломанный накануне, заработал в петербурге\n',
 'лавров и аннан обсудили перспективы конференции по сирии\n',
 'беспорядки произошли на ставрополье, данных о пострадавших нет\n',
 'испания позитивно восприняла доклад мвф\n',
 'биография зубов зубов\n',
 'замглавы мвд рф, курировавший криминалом, снят с должности\n',
 'памятник архитектуры "новая голландия" открывается в петербурге\n',
 'новгородские власти направят 59 млн рублей на выборы губернатора\n',
 'житель тамбовской области насмерть сбил сотрудника дпс\n',
 'футболист зырянов пропустил трени

### 4 Transformer trax

In [2]:
# Default word tokens
PAD_token = 0  # Used for padding short sentences
SOS_token = 1  # Start-of-sentence token
EOS_token = 2  # End-of-sentence token

class Voc:
    def __init__(self, name):
        self.name = name
        self.word2index = {"PAD": PAD_token, "SOS": SOS_token, "EOS": EOS_token}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3  # Count PAD, SOS, EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.index2word[self.num_words] = word
            self.num_words += 1

vocab = Voc('News')

In [3]:
with open('lstm_v2/clear_texts_tokenized_train') as train_texts_file:
    lines = train_texts_file.readlines()

for i in tqdm(range(len(lines))):
    vocab.addSentence(lines[i])

with open('lstm_v2/clear_labels_tokenized_train') as train_labels_file:
    lines = train_labels_file.readlines()

for i in tqdm(range(len(lines))):
    vocab.addSentence(lines[i])

100%|██████████| 963781/963781 [00:03<00:00, 260758.44it/s]


In [8]:
#os.mkdir('trax_transformer/samples')

In [10]:
# записываем каждый сэмпл в отдельный файлик + обрезка и паддинг
TEXT_MAX_LENGTH = 3000
LABEL_MAX_LENGTH = 90

with open('lstm_v2/clear_texts_tokenized_train') as f:
    lines = f.readlines()

with open('lstm_v2/clear_labels_tokenized_train') as f:
    label_lines = f.readlines()

name = 1
for text, label in tqdm(zip(lines, label_lines)):
    text = text.split(' ')
    t_len = len(text)
    if t_len > TEXT_MAX_LENGTH:
        text = text[:TEXT_MAX_LENGTH]
    elif t_len < TEXT_MAX_LENGTH:
        text.extend(["PAD"] * (TEXT_MAX_LENGTH - t_len))
    
    label = label.split(' ')
    l_len = len(label)
    mask = [1] * l_len
    if l_len > LABEL_MAX_LENGTH:
        label = label[:LABEL_MAX_LENGTH]
        mask = mask[:LABEL_MAX_LENGT]
    elif l_len < LABEL_MAX_LENGTH:
        label.extend(["PAD"] * (LABEL_MAX_LENGTH - l_len))
        mask.extend([0] * (LABEL_MAX_LENGTH - l_len))
    
    tokenized_text = []
    for token in text:
        tokenized_text.append(vocab.word2index[token])

    tokenized_label = []
    for token in label:
        tokenized_label.append(vocab.word2index[token])

    output = {'text': tokenized_text, 'mask': mask, 'label': tokenized_label}
    
    with open('trax_transformer/samples/{}'.format(name), 'w') as text_labels_tokens_file:
        json.dump(output, text_labels_tokens_file)
    
    name +=1

963781it [38:00, 422.65it/s]


In [4]:
# даталоадер
def news_inputs(vocab, batch_size, files_dir):
    files_names = os.listdir(files_dir)
    
    while True:
        batch_indexes = np.random.choice(range(1, len(files_names)+1), size=batch_size, replace=False)
        
        texts = []
        labels = []
        mask = []
        for index in batch_indexes:
            with open(os.path.join(files_dir, str(index))) as file:
                sample = json.loads(file.readline())
                texts.append(sample['text'])
                labels.append(sample['label'])
                mask.append(sample['mask'])

        texts = np.stack(texts)
        labels = np.stack(labels)
        mask = np.stack(mask)

        yield texts, labels, mask

In [5]:
copy_inputs = trax.supervised.Inputs(lambda _: news_inputs(vocab, 8, 'trax_transformer/samples'))
data_stream = copy_inputs.train_stream(1)
inputs, target, mask = next(data_stream)

In [6]:
print(inputs.shape)
print(target.shape)
print(mask.shape)
print(len(vocab.word2index))

(8, 3000)
(8, 90)
(8, 90)
74009


In [7]:
# Configure hyperparameters.
gin.parse_config("""
import trax.models
import trax.optimizers
import trax.supervised.inputs
import trax.supervised.trainer_lib

# Parameters for MultifactorSchedule:
# ==============================================================================
MultifactorSchedule.constant = 1.0
MultifactorSchedule.factors = 'constant * linear_warmup * rsqrt_decay'
MultifactorSchedule.warmup_steps = 20000

# Parameters for Adafactor:
# ==============================================================================
Adafactor.beta1 = 0.0
Adafactor.decay_rate = 0.8
Adafactor.clipping_threshold = 1.0
Adafactor.epsilon1 = 1e-30
Adafactor.epsilon2 = 0.001
Adafactor.factored = True
Adafactor.multiply_by_parameter_scale = True


# Parameters for train:
# ==============================================================================
train.optimizer = @trax.optimizers.Adafactor
train.id_to_mask = 0

# Parameters for Transformer:
# ==============================================================================
Transformer.d_model = 512
Transformer.d_ff = 1024
Transformer.dropout = 0.1
Transformer.max_len = 3000
Transformer.mode = 'train'
Transformer.n_heads = 4
Transformer.n_encoder_layers = 2
Transformer.n_decoder_layers = 2
Transformer.input_vocab_size = 74009
""")

In [25]:
#os.mkdir('trax_transformer/model_1')

In [27]:
#shutil.rmtree('trax_transformer/model_1/eval')

In [9]:
output_dir = ('trax_transformer/model_1')
trainer = trax.supervised.Trainer(
    model=trax.models.Transformer,
    loss_fn=trax.layers.CrossEntropyLoss,
    optimizer=trax.optimizers.Adafactor,
    inputs=copy_inputs,
    lr_schedule=trax.lr.MultifactorSchedule,
    output_dir=output_dir,
    has_weights=True)

Model loaded from trax_transformer/model_1/model.pkl at step 159000


In [ ]:
for _ in range(30):
    trainer.train_epoch(n_steps=1000, n_eval_steps=1)


Step  160000: Ran 1000 train steps in 888.60 secs
Step  160000: Evaluation
Step  160000: train                   accuracy |  0.61363637
Step  160000: train                       loss |  2.13586664
Step  160000: train         neg_log_perplexity |  2.13586664
Step  160000: train weights_per_batch_per_core |  88.00000000
Step  160000: eval                    accuracy |  0.48571429
Step  160000: eval                        loss |  2.51269341
Step  160000: eval          neg_log_perplexity |  2.51269341
Step  160000: eval  weights_per_batch_per_core |  70.00000000
Step  160000: Finished evaluation

Step  161000: Ran 1000 train steps in 795.50 secs
Step  161000: Evaluation
Step  161000: train                   accuracy |  0.70652175
Step  161000: train                       loss |  1.43594956
Step  161000: train         neg_log_perplexity |  1.43594956
Step  161000: train weights_per_batch_per_core |  92.00000000
Step  161000: eval                    accuracy |  0.50000000
Step  161000: eval

Step  173000: eval          neg_log_perplexity |  1.77133203
Step  173000: eval  weights_per_batch_per_core |  72.00000000
Step  173000: Finished evaluation

Step  174000: Ran 1000 train steps in 796.52 secs
Step  174000: Evaluation
Step  174000: train                   accuracy |  0.73404253
Step  174000: train                       loss |  1.23178387
Step  174000: train         neg_log_perplexity |  1.23178387
Step  174000: train weights_per_batch_per_core |  94.00000000
Step  174000: eval                    accuracy |  0.48837209
Step  174000: eval                        loss |  2.49915838
Step  174000: eval          neg_log_perplexity |  2.49915838
Step  174000: eval  weights_per_batch_per_core |  86.00000000
Step  174000: Finished evaluation

Step  175000: Ran 1000 train steps in 796.55 secs
Step  175000: Evaluation
Step  175000: train                   accuracy |  0.65979385
Step  175000: train                       loss |  1.73388553
Step  175000: train         neg_log_perplexit

In [16]:
# os.mkdir('trax_transformer/samples_test/')

In [ ]:
# записываем каждый сэмпл в отдельный файлик + обрезка и паддинг
TEXT_MAX_LENGTH = 3000
LABEL_MAX_LENGTH = 90
cancels = 0

with open('lstm_v2/clear_texts_tokenized_test') as f:
    lines = f.readlines()

with open('lstm_v2/clear_labels_tokenized_test') as f:
    label_lines = f.readlines()

name = 1
for text, label in tqdm(zip(lines, label_lines)):
    try:
        text = text.split(' ')
        t_len = len(text)
        if t_len > TEXT_MAX_LENGTH:
            text = text[:TEXT_MAX_LENGTH]
        elif t_len < TEXT_MAX_LENGTH:
            text.extend(["PAD"] * (TEXT_MAX_LENGTH - t_len))

        label = label.split(' ')
        l_len = len(label)
        mask = [1] * l_len
        if l_len > LABEL_MAX_LENGTH:
            label = label[:LABEL_MAX_LENGTH]
            mask = mask[:LABEL_MAX_LENGT]
        elif l_len < LABEL_MAX_LENGTH:
            label.extend(["PAD"] * (LABEL_MAX_LENGTH - l_len))
            mask.extend([0] * (LABEL_MAX_LENGTH - l_len))

        tokenized_text = []
        for token in text:
            tokenized_text.append(vocab.word2index[token])

        tokenized_label = []
        for token in label:
            tokenized_label.append(vocab.word2index[token])

        output = {'text': tokenized_text, 'mask': mask, 'label': tokenized_label}

        with open('trax_transformer/samples_test/{}'.format(name), 'w') as text_labels_tokens_file:
            json.dump(output, text_labels_tokens_file)

        name +=1
    
    except:
        cancels +=1
        continue

In [13]:
#функция для получения предсказаний модели
def index_to_token(x):
    return vocab.index2word[x]

def validate_inputs(vocab, batch_size, files_dir, model, predict_dir, true_dir):
    files_names = os.listdir(files_dir)
    files_names = np.arange(1, len(files_names) + 1)
    back_to_tokens = np.vectorize(index_to_token)
    
    batch_indexes = files_names[::batch_size]
    for i in tqdm(range(len(batch_indexes) - 1)):
        batch_range = range(batch_indexes[i], batch_indexes[i+1])
        texts = []
        labels = []
        
        for index in batch_range:
            with open(os.path.join(files_dir, str(index))) as file:
                sample = json.loads(file.readline())
                texts.append(sample['text'])
                labels.append(sample['label'])
     
        texts = np.stack(texts)
        labels = np.stack(labels)

        predict = model((texts, labels))[0].argmax(axis=-1)
        mask = predict[:, 1:] - predict[:, :-1]

        for j, sentence in enumerate(predict):
            stop_index = np.argmax(mask[j] == 0) + 1
            to_write = ' '.join(back_to_tokens(sentence[:stop_index]))
            to_write = re.sub(r'\n|\r', '', to_write) + '\n'
            
            stop_index_labels = np.argmax(labels[j] == 0)
            true = ' '.join(back_to_tokens(labels[j][:stop_index_labels]))

            with open(predict_dir, 'a') as file_prediction:
                file_prediction.write(to_write)
            
            with open(true_dir, 'a') as file_true:
                file_true.write(true)

#### 4.1 тестирование модели

In [12]:
model = trax.models.Transformer(mode='eval')
model.init_from_file('trax_transformer/model_1/model.pkl')

In [13]:
validate_inputs(vocab, 8, 'trax_transformer/samples_test', model,
                'trax_transformer/model_1/prediction',
                'trax_transformer/model_1/true')

100%|██████████| 2492/2492 [26:26<00:00,  1.63it/s]


In [18]:
BPE = pyonmttok.Tokenizer("conservative", bpe_model_path="model-50k_with_joiner")

In [20]:
#os.remove('trax_transformer/model_1/true_detokenized')

In [21]:
BPE.detokenize_file(input_path='trax_transformer/model_1/prediction',
                    output_path='trax_transformer/model_1/prediction_detokenized')

In [22]:
BPE.detokenize_file(input_path='trax_transformer/model_1/true',
                    output_path='trax_transformer/model_1/true_detokenized')

#### Очистка от пустых предсказанных заголовков

In [23]:
with open('trax_transformer/model_1/true_detokenized') as f:
    true = f.readlines()
with open('trax_transformer/model_1/prediction_detokenized') as f:
    prediction = f.readlines()

In [67]:
cancels = []
for i, sentence in enumerate(prediction):
    if len(sentence) < 5:
        cancels.append(i)

In [68]:
len(cancels)

297

In [66]:
# os.remove('trax_transformer/model_1/true_for_metrics')

In [69]:
for i, line in enumerate(prediction):
    if i not in cancels:
        with open('trax_transformer/model_1/prediction_for_metrics', 'a') as f1:
            f1.write(line)

In [70]:
for i, line in enumerate(true):
    if i not in cancels:
        with open('trax_transformer/model_1/true_for_metrics', 'a') as f1:
            f1.write(line)

#### Метрики

In [73]:
from rouge import FilesRouge
files_rouge_detoc = FilesRouge('trax_transformer/model_1/prediction_for_metrics',
                              'trax_transformer/model_1/true_for_metrics')
scores_detoc = files_rouge_detoc.get_scores(avg=True, ignore_empty=True)

In [74]:
scores_detoc

{'rouge-1': {'f': 0.4290884118256301,
  'p': 0.4632726660197333,
  'r': 0.4321201598256151},
 'rouge-2': {'f': 0.2135105618761511,
  'p': 0.20826609730102505,
  'r': 0.2292366333617198},
 'rouge-l': {'f': 0.4029890042208497,
  'p': 0.45219391395157066,
  'r': 0.42164615234501285}}

In [75]:
with open('trax_transformer/model_1/prediction_for_metrics') as f:
    prediction_samples = f.readlines()
    
with open('trax_transformer/model_1/true_for_metrics') as f:
    true_samples = f.readlines()

In [90]:
true_samples[200]

'два здания обрушились в италии из-за взрыва газа, 10 человек ранены\n'

In [91]:
prediction_samples[200]

'два здания обрушились в горах, взрыва газа, есть человек пострадали ранены\n'

### 5. Reformer (reversible only)

In [2]:
# Default word tokens
PAD_token = 0  # Used for padding short sentences
#SOS_token = 1  # Start-of-sentence token
EOS_token = 1  # End-of-sentence token

class Voc:
    def __init__(self, name):
        self.name = name
        self.word2index = {"PAD": PAD_token, "EOS": EOS_token}
        self.index2word = {PAD_token: "PAD", EOS_token:"EOS"}
        self.num_words = 2  # Count PAD, EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.index2word[self.num_words] = word
            self.num_words += 1

vocab = Voc('News')

In [3]:
with open('lstm_v2/clear_texts_tokenized_train') as train_texts_file:
    lines = train_texts_file.readlines()

for i in tqdm(range(len(lines))):
    vocab.addSentence(lines[i])

with open('lstm_v2/clear_labels_tokenized_train') as train_labels_file:
    lines = train_labels_file.readlines()

for i in tqdm(range(len(lines))):
    vocab.addSentence(lines[i])

100%|██████████| 963781/963781 [00:03<00:00, 286434.17it/s]


In [11]:
# os.mkdir('Reformer_reversible/samples')

In [12]:
TEXT_MAX_LENGTH = 3000
LABEL_MAX_LENGTH = 90

with open('lstm_v2/clear_texts_tokenized_train') as f:
    lines = f.readlines()

with open('lstm_v2/clear_labels_tokenized_train') as f:
    label_lines = f.readlines()

name = 1
for text, label in tqdm(zip(lines, label_lines)):
    text = text.split(' ')
    t_len = len(text)
    if t_len > TEXT_MAX_LENGTH:
        text = text[:TEXT_MAX_LENGTH]
    elif t_len < TEXT_MAX_LENGTH:
        text.extend(["PAD"] * (TEXT_MAX_LENGTH - t_len))
    
    label = label.split(' ')
    l_len = len(label)
    mask = [1] * l_len
    if l_len > LABEL_MAX_LENGTH:
        label = label[:LABEL_MAX_LENGTH]
        label[-1] = "EOS"
        mask = mask[:LABEL_MAX_LENGTH]
    elif l_len < LABEL_MAX_LENGTH:
        label.append("EOS")
        mask.append(1)
        label.extend(["PAD"] * (LABEL_MAX_LENGTH - l_len - 1))
        mask.extend([0] * (LABEL_MAX_LENGTH - l_len - 1))
    else:
        label[-1] = "EOS"
    
    tokenized_text = []
    for token in text:
        tokenized_text.append(vocab.word2index[token])

    tokenized_label = []
    for token in label:
        tokenized_label.append(vocab.word2index[token])

    output = {'text': tokenized_text, 'mask': mask, 'label': tokenized_label}
    
    with open('Reformer_reversible/samples/{}'.format(name), 'w') as text_labels_tokens_file:
        json.dump(output, text_labels_tokens_file)
    
    name +=1

963781it [38:26, 417.90it/s]


In [4]:
def news_inputs(vocab, batch_size, files_dir):
    files_names = os.listdir(files_dir)
    
    while True:
        batch_indexes = np.random.choice(range(1, len(files_names)+1), size=batch_size, replace=False)
        
        texts = []
        labels = []
        mask = []
        for index in batch_indexes:
            with open(os.path.join(files_dir, str(index))) as file:
                sample = json.loads(file.readline())
                texts.append(sample['text'])
                labels.append(sample['label'])
                mask.append(sample['mask'])

        texts = np.stack(texts)
        labels = np.stack(labels)
        mask = np.stack(mask)

        yield texts, labels, mask

In [5]:
copy_inputs = trax.supervised.Inputs(lambda _: news_inputs(vocab, 8, 'Reformer_reversible/samples'))
data_stream = copy_inputs.train_stream(1)
inputs, target, mask = next(data_stream)

In [6]:
print(inputs.shape)
print(target.shape)
print(mask.shape)

(8, 3000)
(8, 90)
(8, 90)


In [7]:
len(vocab.index2word)

74008

In [8]:
gin.parse_config("""
import trax.models
import trax.optimizers
import trax.supervised.inputs
import trax.supervised.trainer_lib
# Parameters for _is_jit_init:
# ==============================================================================
_is_jit_init.value = True
# Parameters for _jit_predict_fn:
# ==============================================================================
_jit_predict_fn.jit = True
# Parameters for _jit_update_fn
# ==============================================================================
_jit_update_fn.jit = True
# Parameters for Adafactor:
# ==============================================================================
Adafactor.beta1 = 0.0
Adafactor.decay_rate = 0.8
Adafactor.clipping_threshold = 1.0
Adafactor.epsilon1 = 1e-30
Adafactor.epsilon2 = 0.001
Adafactor.factored = True
Adafactor.multiply_by_parameter_scale = True
# Parameters for backend:
# ==============================================================================
backend.name = 'jax'
# Parameters for EncDecAttention:
# ==============================================================================
EncDecAttention.masked = True
EncDecAttention.n_parallel_heads = None
EncDecAttention.use_python_loop = False
EncDecAttention.use_reference_code = False

# Parameters for MultifactorSchedule:
# ==============================================================================
MultifactorSchedule.constant = 1.0
MultifactorSchedule.factors = 'constant * linear_warmup * rsqrt_decay'
MultifactorSchedule.warmup_steps = 3000
# Parameters for Reformer:
# ==============================================================================
Reformer.d_ff = 1024
Reformer.d_model = 512
Reformer.dropout = 0.1
Reformer.ff_activation = @trax.layers.Relu
Reformer.ff_dropout = 0.1
Reformer.input_vocab_size = 74008
Reformer.max_len = 3000
Reformer.n_decoder_layers = 6
Reformer.n_encoder_layers = 6
Reformer.n_heads = 4
#Reformer.output_vocab_size

# Parameters for SelfAttention
# ==============================================================================
# SelfAttention.causal = False
# SelfAttention.chunk_len = None
# SelfAttention.masked = False
# SelfAttention.n_chunks_after = 0
# SelfAttention.n_chunks_before = 0
# SelfAttention.n_parallel_heads = None
# SelfAttention.predict_drop_len = 64
# SelfAttention.predict_mem_len = 192
# SelfAttention.share_qk = False
# SelfAttention.use_python_loop = False
# SelfAttention.use_reference_code = False

# Parameters for train:
# ==============================================================================
#train.checkpoint_highest = None
#train.checkpoint_lowest = None
#train.checkpoints_at = None
#train.eval_frequency = 1000
#train.eval_steps = 10
""")

In [ ]:
output_dir = 'Reformer_reversible/model_2'

trainer = trax.supervised.Trainer(
    model=trax.models.Reformer,
    loss_fn=trax.layers.CrossEntropyLoss,
    optimizer=trax.optimizers.Adafactor,
    inputs=copy_inputs,
    lr_schedule=trax.lr.MultifactorSchedule,
    output_dir=output_dir,
    id_to_mask=0,
    has_weights=True)

In [15]:
for _ in range(15):
    trainer.train_epoch(n_steps=1000, n_eval_steps=1)


Step  111000: Ran 1000 train steps in 3105.33 secs
Step  111000: Evaluation
Step  111000: train                   accuracy |  0.40217391
Step  111000: train                       loss |  3.81247330
Step  111000: train         neg_log_perplexity |  3.81247330
Step  111000: train          sequence_accuracy |  0.00000000
Step  111000: train weights_per_batch_per_core |  92.00000000
Step  111000: eval                    accuracy |  0.40000001
Step  111000: eval                        loss |  3.64806557
Step  111000: eval          neg_log_perplexity |  3.64806557
Step  111000: eval           sequence_accuracy |  0.00000000
Step  111000: eval  weights_per_batch_per_core |  90.00000000
Step  111000: Finished evaluation

Step  112000: Ran 1000 train steps in 2883.80 secs
Step  112000: Evaluation
Step  112000: train                   accuracy |  0.46296296
Step  112000: train                       loss |  3.51094055
Step  112000: train         neg_log_perplexity |  3.51094055
Step  112000: tra

Step  122000: train          sequence_accuracy |  0.00000000
Step  122000: train weights_per_batch_per_core |  112.00000000
Step  122000: eval                    accuracy |  0.45833334
Step  122000: eval                        loss |  2.80004311
Step  122000: eval          neg_log_perplexity |  2.80004311
Step  122000: eval           sequence_accuracy |  0.00000000
Step  122000: eval  weights_per_batch_per_core |  96.00000000
Step  122000: Finished evaluation

Step  123000: Ran 1000 train steps in 2885.01 secs
Step  123000: Evaluation
Step  123000: train                   accuracy |  0.53846157
Step  123000: train                       loss |  2.62778187
Step  123000: train         neg_log_perplexity |  2.62778187
Step  123000: train          sequence_accuracy |  0.00000000
Step  123000: train weights_per_batch_per_core |  104.00000000
Step  123000: eval                    accuracy |  0.49367091
Step  123000: eval                        loss |  3.09498739
Step  123000: eval          ne

In [11]:
for _ in range(5):
    trainer.train_epoch(n_steps=1000, n_eval_steps=1)


Step  228738: Ran 1000 train steps in 2939.15 secs
Step  228738: Evaluation
Step  228738: train                   accuracy |  0.63265306
Step  228738: train                       loss |  1.76499438
Step  228738: train         neg_log_perplexity |  1.76499438
Step  228738: train          sequence_accuracy |  0.00000000
Step  228738: train weights_per_batch_per_core |  98.00000000
Step  228738: eval                    accuracy |  0.61111110
Step  228738: eval                        loss |  2.07283592
Step  228738: eval          neg_log_perplexity |  2.07283592
Step  228738: eval           sequence_accuracy |  0.00000000
Step  228738: eval  weights_per_batch_per_core |  90.00000000
Step  228738: Finished evaluation

Step  229738: Ran 1000 train steps in 2705.43 secs
Step  229738: Evaluation
Step  229738: train                   accuracy |  0.54022986
Step  229738: train                       loss |  2.19249463
Step  229738: train         neg_log_perplexity |  2.19249463
Step  229738: tra